В этом файле представлены данные по поступлению денежных средств от клиентов компании N за покупку ее услуг.

Имеющиеся поля:
* client_id - ID клиента;
* date – дата получения денежных средств;
* sum – сумма денежных средств;
* status – статус оплаты;
* sale – менеджер, заключивший сделку;
* new/current – статус сделки;
* document – наличие оригинала подписанного договора с клиентом;
* receiving_date – дата получения оригинала договора.

В данных могут быть пропущенные значения.

Вопросы:
1. Вычислите общую выручку за июль 2021 по тем сделкам, приход денежных средств которых не просрочен.
2. Как изменилась выручка компании в августе 2021 по сравнению с маем 2021? Проиллюстрируйте графиком.
3. Кто из менеджеров привлек для компании больше всего денежных средств в сентябре 2021?

Задание:
За каждую заключенную сделку менеджер получает бонус, который рассчитывается следующим образом.
1) За новые сделки менеджер получает 7 % от суммы, при условии, что статус оплаты «ОПЛАЧЕНО», а также имеется оригинал подписанного договора с клиентом.
2) За текущие сделки менеджер получает 5 % от суммы, если она больше 10 тыс., и 3 % от суммы, если меньше. При этом статус оплаты может быть любым, кроме «ПРОСРОЧЕНО», а также необходимо наличие оригинала подписанного договора с клиентом.
* Вычислите сколько бонусов каждый из менеджеров должен был получить в июне 2021.

In [1]:
# Импортируем рабочие библиотеки
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [111]:
# Загрузим данные
df = pd.read_csv('test_data_1.csv', sep=',', parse_dates=['receiving_date'])
df.head()

,client_id,date,sum,status,sale,new/current,document,receiving_date
0,6,мая 2021,"11 693,70",ВНУТРЕННИЙ,-,-,-,-
1,14,мая 2021,"5 452,70",ОПЛАЧЕНО,Петрова,новая,оригинал,26.07.21
2,15,мая 2021,"13 991,70",ОПЛАЧЕНО,Иванов,текущая,оригинал,08.06.21
3,16,мая 2021,"719,7",ОПЛАЧЕНО,Иванов,текущая,оригинал,08.06.21
4,18,мая 2021,"4 253,70",ОПЛАЧЕНО,Иванов,текущая,оригинал,07.07.21


In [112]:
# Оценим что прочитано корректно
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 8 columns):
client_id         724 non-null int64
date              724 non-null object
sum               724 non-null object
status            724 non-null object
sale              724 non-null object
new/current       724 non-null object
document          709 non-null object
receiving_date    607 non-null object
dtypes: int64(1), object(7)
memory usage: 45.4+ KB


In [115]:
# Исправим недочеты
df['sum'] = df['sum'].str.replace('\xa0', '').str.replace(',', '.').astype(float)
df.rename({'sum': 'summa'}, axis=1, inplace=True)

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 8 columns):
client_id         724 non-null int64
date              724 non-null object
summa             724 non-null float64
status            724 non-null object
sale              724 non-null object
new/current       724 non-null object
document          709 non-null object
receiving_date    607 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 45.4+ KB


In [309]:
# Вычислите общую выручку за июль 2021 по тем сделкам, приход денежных средств которых не просрочен.
df_july = df.query('date == "июля 2021" and status != "ПРОСРОЧЕНО"')
print(f'Общая выручка за июль 2021 составляет {round(df_july.summa.sum())}')

Общая выручка за июль 2021 составляет 859896.0


In [139]:
# Как изменилась выручка компании в августе 2021 по сравнению с маем 2021?
df_changed = df.query('date == "мая 2021" or date == "августа 2021"')
df_changed = df_changed.groupby('date').sum().sort_values('summa')
df_changed

,client_id,summa
date,,
мая 2021,17255,809572.19
августа 2021,22517,850435.79


In [310]:
# Проиллюстрируем графиком
px.bar(df_changed, x=df_changed.index, y='summa', color='summa')

In [178]:
# Кто из менеджеров привлек для компании больше всего денежных средств в сентябре 2021?
df_sep = df.query('date == "сентября 2021"')
manager = df_sep.groupby('sale').sum().sort_values('summa', ascending=False).head(1)
print(f'Менеджер {manager.index[0]} привлек для компании {round(manager.summa[0])} денежных средств в сентябре 2021')

Менеджер Смирнов привлек для компании 221526.0 денежных средств в сентябре 2021


За каждую заключенную сделку менеджер получает бонус, который рассчитывается следующим образом. <br>
1) За новые сделки менеджер получает 7 % от суммы, при условии, что статус оплаты «ОПЛАЧЕНО», а также имеется оригинал подписанного договора с клиентом. <br>
2) За текущие сделки менеджер получает 5 % от суммы, если она больше 10 тыс., и 3 % от суммы, если меньше. При этом статус оплаты может быть любым, кроме «ПРОСРОЧЕНО», а также необходимо наличие оригинала подписанного договора с клиентом. 
- Вычислите сколько бонусов каждый из менеджеров должен был получить в июне 2021.<br>

In [311]:
# Сгруппируем данные по дате и документу.
df_june = df.query('date == "июня 2021" and document == "оригинал"')

bonuce_for_manager = {}

# Пройдем циклом по каждому менеджеру и посчитаем сколько бонусов получил каждый.
for manager in df_june.sale.unique():
    df_manager = df_june[df_june['sale'] == manager]
    sort = df_manager[(df_manager['new/current'] == 'новая') & (df_manager['status'] == 'ОПЛАЧЕНО')]
    sort1 = df_manager[(df_manager['new/current'] == 'текущая') & (df_manager['status'] != 'ПРОСРОЧЕНО')]
    
    sort['cal'] = sort['summa'] * 0.07
    a = sort.cal.sum()
    
    sort1['cal'] = np.where(sort1['summa'] > 10000,  sort1['summa'] * 0.05, sort1['summa'] * 0.03)
    b = sort1.cal.sum()
    
    bonuce_for_manager[manager] = round(a + b, 2)

bonuce_for_manager

C:\CONDA\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\CONDA\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



{'Петрова': 8904.47,
 'Иванов': 4300.37,
 'Кузнецова': 3427.48,
 'Андреев': 3499.94,
 'Филимонова': 1654.09,
 'Смирнов': 5854.74,
 'Селиванов': 3367.38,
 'Васильев': 932.68,
 'Соколов': 269.21}